In [69]:
from dotenv import load_dotenv
load_dotenv()

import os

import numpy as np
from pixell import enmap, enplot, reproject
import glob
import matplotlib.pyplot as plt
import emcee, corner
from astropy.io import fits
import sys
from astropy import units as u, constants as const
sys.path.insert(0, '../src')
import sys
import warnings
import os
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
import yaml
import itertools
from pixell import enmap

import bandpass as bp
import covariance as cov
import model
import utils as ut

import SZpack as SZ

from astropy.coordinates import SkyCoord
import astropy.units as u

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
emap = enmap.read_map("/home/gill/research/ACT/bridge/a399_a401/data/act_no_reproj/act_cut_dr6v2_pa5_f098_4way_coadd_ivar.fits")

ax = plt.subplot(projection=emap.wcs)
ax.imshow(emap, origin='lower', cmap='gray')
plt.xlabel('RA')
plt.ylabel('Dec')

In [ ]:
# change plt parameters to tex font
plt.rcParams['text.usetex'] = 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
from astropy.visualization import ImageNormalize

# use latex for the plot
plt.rc('text', usetex=True)
plt.rc('font', family='sans-serif', size=25)

# Compute ZScale limits
# Create the plot with WCS projection
plt.figure(figsize=(8, 8))

flux_factor = ut.flux_factor("pa5", 98)

emap_jy = emap / flux_factor**2

ax = plt.subplot(projection=emap_jy.wcs)

norm = ImageNormalize(emap_jy, interval=ZScaleInterval())

# Display the map with ZScale normalization
im = ax.imshow(emap_jy, origin='lower', cmap='viridis', norm=norm, interpolation='none')

ra = ax.coords[0]
dec = ax.coords[1]

# Set labels
ra.set_axislabel('Right ascension')
dec.set_axislabel('Declination')

# Ensure tick labels are in float degrees
ra.set_major_formatter('d')
dec.set_major_formatter('d')

ax.invert_xaxis()
plt.colorbar(im,fraction=0.046, pad=0.04, label='(Jy/sr)$^{-2}$')

plt.savefig('../plots/ivar_map.eps', bbox_inches='tight', dpi=300, format='eps')
plt.show()


In [70]:
def converge_plot(sampler, labels):
    plt.figure(figsize=(12, 8))
    for idx in range(len(labels)):
        chain = sampler.get_chain()[:, :, idx].T

        N = np.exp(np.linspace(np.log(10**2), np.log(chain.shape[1]), 10)).astype(int)
        gw2010 = np.empty(len(N))
        new = np.empty(len(N))
        for i, n in enumerate(N):
            gw2010[i] = ut.autocorr_gw2010(chain[:, :n])
            new[i] = ut.autocorr_new(chain[:, :n])

        plt.loglog(N, new, "o-", label=labels[idx])
        plt.xlabel("number of samples, $N$")
        plt.ylabel(r"$\tau$ estimates")
            
    plt.plot(N, N / 50.0, "--k", label=r"$\tau = N/50$")
    plt.legend(fontsize=8)
    plt.grid()
    plt.show()
    
def plotter_sim(filename, target, p_range, plot_contours=False, fit_dust=True,
                plot_converge=False, burnin=1, 
                plot_samples=True, truths=None,
                thin=1, plot_maps=False, cf_path="cf_auto_all.yaml"):
    
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    
    # Load interpolated function dictionary
    # interp_f_dict = ut.load_interp_funcs()
    
    dire = f"/home/a/ahincks/gillajay/scratch/bridge"
    
    sampler = emcee.backends.HDFBackend(filename)
        
    cf = ut.get_config_file(f'{cf_path}')
        
    labels = [r'$RA_{\rm A401}$', 
              r'$DEC_{\rm A401}$', 
              r'$\beta_{\rm A401}$', 
              r'$r_{\rm c, A401}$ [$^{\prime}$]', 
              r'$e_{\rm A401}$', 
              r'$\theta_{\rm A401}$', 
              r'$\tau_{\rm A401}$', 
              r'$T_{\rm e, A401}$', 
              r'$A_{\rm D, A401}$',
              
              r'$RA_{\rm A399}$', 
              r'$DEC_{\rm A399}$', 
              r'$\beta_{\rm A399}$', 
              r'$r_{\rm c, A399}$ [$^{\prime}$]', 
              r'$e_{\rm A399}$', 
              r'$\theta_{\rm A399}$', 
              r'$\tau_{\rm A399}$', 
              r'$T_{\rm e, A399}$', 
              r'$A_{\rm D, A399}$',
            
              r"$RA_{\rm fil}",
              r"$DEC_{\rm fil}",
              r"$L_{\rm fil}$",
              r"$W_{\rm fil}$",
              r'$\tau_{\rm fil}$',
              r'$T_{\rm e, fil}$',
              r'$A_{\rm D, fil}$',
              
              r"$v_{r, avg}$"]
    
    labels_mode = [r"$RA_{\rm fil} [$^{\circ}$]",
              r"$DEC_{\rm fil} [$^{\circ}$]",
              r"$L_{\rm fil}$ [$^{\prime}$]",
              r"$W_{\rm fil}$ [$^{\prime}$]",
              r'$\tau_{\rm fil}$',
              r'$T_{\rm e, fil}$ [keV]',
              r'$A_{\rm D, fil}$ [Jy/sr]']
    
    #labels_mode = [r"$DEC_{\rm fil}", r'$A_{\rm D, fil}$']
    
    labels_a401 = [r'RA$_{\rm A401}$ [$^{\circ}$]', 
              r'DEC$_{\rm A401}$ [$^{\circ}$]', 
              r'$\beta_{\rm A401}$', 
              r'$r_{\rm c, A401}$ [$^{\prime}$]', 
              r'$e_{\rm A401}$', 
              r'$\theta_{\rm A401}$ [$^{\circ}$]', 
              r'$\tau_{\rm A401}$', 
              r'$T_{\rm e, A401}$ [keV]', 
              r'$A_{\rm D, A401}$ [Jy/sr]',
              r"$v_{\rm r, avg}$ [km/s]"]
    
    labels_a399 = [r'$RA_{\rm A399}$ [$^{\circ}$]', 
              r'$DEC_{\rm A399}$ [$^{\circ}$]', 
              r'$\beta_{\rm A399}$', 
              r'$r_{\rm c, A399}$ [$^{\prime}$]', 
              r'$e_{\rm A399}$', 
              r'$\theta_{\rm A399}$ [$^{\circ}$]', 
              r'$\tau_{\rm A399}$', 
              r'$T_{\rm e, A399}$ [keV]', 
              r'$A_{\rm D, A399}$ [Jy/sr]',
              r"$v_{\rm r, avg}$ [km/s]"]
    
    labels_fil =  [r"$RA_{\rm fil}$ [$^{\circ}$]",
              r"$DEC_{\rm fil}$ [$^{\circ}$]",
              r"$L_{\rm fil}$ [$^{\prime}$]",
              r"$W_{\rm fil}$ [$^{\prime}$]",
              r'$\tau_{\rm fil}$',
              r'$T_{\rm e, fil}$ [keV]',
              r'$A_{\rm D, fil}$ [Jy/sr]',
              
              r"$v_{\rm r, avg}$ [km/s]"]
    
            
    region = ut.get_region(cf['region_center_ra'], 
                           cf['region_center_dec'], 
                           cf['region_width'])
    
    dire_data = "/home/gill/research/ACT/bridge/data_paper/data/data/act_no_reproj"
    data_ref = enmap.read_map(f"{dire_data}/act_cut_dr6v2_pa5_f098_4way_coadd_map_srcfree.fits", 
                              box=region)
    
    data_wcs = data_ref.wcs
    
    idx_ra_401_pix = 0
    idx_dec_401_pix = 1
    
    idx_ra_399_pix = 9
    idx_dec_399_pix = 10
    
    idx_ra_fil_pix = 18
    idx_dec_fil_pix = 19
    
    samples = sampler.get_chain(discard=burnin, flat=True, thin=thin)
    
    print("\nNumber of iterations: {:.0f}".format(samples.shape[0] / sampler.shape[0]))

    
    a401_ra_pix = samples[:, idx_ra_401_pix]
    a401_dec_pix = samples[:, idx_dec_401_pix]
    
    a399_ra_pix = samples[:, idx_ra_399_pix]
    a399_dec_pix = samples[:, idx_dec_399_pix]
    
    fil_ra_pix = samples[:, idx_ra_fil_pix]
    fil_dec_pix = samples[:, idx_dec_fil_pix]
    
    ra_a401 = data_ref.wcs.celestial.wcs_pix2world(a401_ra_pix, a401_dec_pix, 0)[0]
    dec_a401 = data_ref.wcs.celestial.wcs_pix2world(a401_ra_pix, a401_dec_pix, 0)[1]
      
    ra_a399 = data_ref.wcs.celestial.wcs_pix2world(a399_ra_pix, a399_dec_pix, 0)[0]
    dec_a399 = data_ref.wcs.celestial.wcs_pix2world(a399_ra_pix, a399_dec_pix, 0)[1]
        
    ra_fil = data_ref.wcs.celestial.wcs_pix2world(fil_ra_pix, fil_dec_pix, 0)[0]
    dec_fil = data_ref.wcs.celestial.wcs_pix2world(fil_ra_pix, fil_dec_pix, 0)[1]

    # print("\nA401")
    # print("RA: {:.6f}, +/- {:.6f}".format(np.mean(ra_a401), np.std(ra_a401)))
    # print("DEC: {:.6f}, +/- {:.6f}".format(np.mean(dec_a401), np.std(dec_a401)))

    # print("\nA399")
    # print("RA: {:.6f}, +/- {:.6f}".format(np.mean(ra_a399), np.std(ra_a399)))
    # print("DEC: {:.6f}, +/- {:.6f}".format(np.mean(dec_a399), np.std(dec_a399)))

    # print("\nFilament")
    # print("RA: {:.6f}, +/- {:.6f}".format(np.mean(ra_fil), np.std(ra_fil)))
    # print("DEC: {:.6f}, +/- {:.6f}".format(np.mean(dec_fil), np.std(dec_fil)))
    
    samples[:, idx_ra_401_pix] = ra_a401
    samples[:, idx_dec_401_pix] = dec_a401

    samples[:, idx_ra_399_pix] = ra_a399
    samples[:, idx_dec_399_pix] = dec_a399
    
    samples[:, idx_ra_fil_pix] = ra_fil
    samples[:, idx_dec_fil_pix] = dec_fil
    
    samples[:, 20] *= 0.5
    samples[:, 21] *= 0.5
    
    columns_a401 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 25]
    columns_a399 = [9, 10, 11, 12, 13, 14, 15, 16, 17, 25]
    columns_fil = [18, 19, 20, 21, 22, 23, 24, 25]
    
    samples_short = samples[:, columns_a401]
    
    theta = []
    
    for idx, label in enumerate(labels):
        mcmc_run = np.percentile(samples[:, idx], [16, 50, 84])
        err = .5 * (mcmc_run[2] - mcmc_run[0])
                
        #if label in labels_mode:
        data = samples[:, idx]

        # Calculate bin width using Freedman-Diaconis rule
        n = len(data)
        iqr = np.percentile(data, 75) - np.percentile(data, 25)
        bin_width = 2 * iqr * n**(-1/3)

        # Calculate number of bins using bin width
        num_bins = int((np.max(data) - np.min(data)) / bin_width)

        # Create histogram
        hist, bin_edges = np.histogram(data, bins=num_bins, density=True)

        # Find bin with maximum value (estimate of mode)
        mode_bin = np.argmax(hist)
        mode_value = 0.5 * (bin_edges[mode_bin] + bin_edges[mode_bin + 1])

        mean = mode_value
        
#         else:
#             mean = mcmc_run[1]
        
        theta.append(mean)

        print("{}: {:.6f}, +/- {:.6f}".format(label, mean, err))

    truths = np.array(theta)
    #truths = None
    
    if plot_contours: 
        levels = (1 - np.exp(-0.5), 1 - np.exp(-2) )
        if truths is not None:
            figure = corner.corner(samples, 
                                   labels=labels,
                                   truths=truths,
                                   #range=p_range,                                  
                                   show_titles=False, 
                                   plot_datapoints=0,
                                   title_fmt=".3f")
            
            figure.subplots_adjust(right=3,top=3)
            
            ax = figure.gca()
            # plt.text(-5, 5, 'Preliminary', transform=ax.transAxes,
            #     fontsize=300, color='gray', alpha=0.2,
            #     ha='center', va='center', rotation=30)
            
            for ax in figure.get_axes():
              #ax.tick_params(axis='both', which='major', labelsize=14)
              #ax.tick_params(axis='both', which='minor', labelsize=12)    
                ax.tick_params(axis='both', labelsize=40)
                ax.xaxis.label.set_size(60)
                ax.yaxis.label.set_size(60)
            
            
            # save figure
            figure.savefig("../plots/corner_a401.pdf", dpi=500, bbox_inches='tight')

            #figure.savefig("plots/corner_fil.pdf", dpi=500, bbox_inches='tight')
        else:
            figure = corner.corner(samples, 
                       labels=labels,
                       #quantiles=[0.16, 0.5, 0.84],
                       show_titles=False, 
                       #range=p_range,
                       plot_datapoints=False,
                       title_fmt=".3f")
            figure.subplots_adjust(right=3,top=3)
                        
            ax = figure.gca()
            # plt.text(-5, 5, 'Preliminary', transform=ax.transAxes,
            #     fontsize=300, color='gray', alpha=0.2,
            #     ha='center', va='center', rotation=30)
            
            for ax in figure.get_axes():
              #ax.tick_params(axis='both', which='major', labelsize=14)
              #ax.tick_params(axis='both', which='minor', labelsize=12)    
                ax.tick_params(axis='both', labelsize=40)
                ax.xaxis.label.set_size(60)
                ax.yaxis.label.set_size(60)
                
        plt.show()

    if plot_converge:
        print("Convergence plot")
        converge_plot(sampler, labels_a401)

In [71]:
target = "bridge"

p_range_a401 = [(44.736, 44.746),
               (13.57, 13.59),
               (0.9, 1.4),
               (2, 4.8),
               (1, 1.7),
               (80, 140),
               (.0078, .014),
               (7.5, 9.5),
               (-1e5, 501400.053986 * 2.5),
               (1000, 3200)]

p_range_a399 = [(44.45, 44.476),
               (13.025, 13.052),
               (.9, 1.8),
               (2, 7.5),
               (.6, 1.6),
               (0, 180),
               (.0038, .0072),
               (6.8, 7.7),
               (-50e3, 700e3),
               (-2500, 2500)]

    
#     labels_fil =  [r"$RA_{\rm fil}",
#               r"$DEC_{\rm fil}",
#               r"$L_{\rm fil}$",
#               r"$W_{\rm fil}$",
#               r'$\tau_{\rm fil}$',
#               r'$T_{\rm e, fil}$',
#               r'$A_{\rm D, fil}$',
              
#               r"$v_{r, avg}$"]

p_range_fil = [(44.55, 44.8),
               (13.2, 13.5),
               (8, 20),
               (10, 18),
               (.0008, 0.0021),
               (5.2, 7.5),
               (-25e3, 200e3),
               (1000, 4500)]

model_tot= plotter_sim(filename=f"/home/gill/research/ACT/bridge/results/ajay/case46/chain.h5", 
                       target=f"{target}",
                       plot_maps=1,
                       burnin=200,
                       thin=1,
                       fit_dust=1,
                       plot_contours=1,
                       p_range=p_range_a401,
                       plot_samples=0,
                       cf_path="/home/gill/research/ACT/multi-freq-bridge/configs/case46_ajay.yaml",
                       plot_converge=0)


Number of iterations: 3788
$RA_{\rm A401}$: 44.741066, +/- 0.001706
$DEC_{\rm A401}$: 13.580224, +/- 0.002194
$\beta_{\rm A401}$: 1.113422, +/- 0.078496
$r_{\rm c, A401}$ [$^{\prime}$]: 3.538906, +/- 0.377527
$e_{\rm A401}$: 1.295297, +/- 0.101755
$\theta_{\rm A401}$: 110.401269, +/- 6.217432
$\tau_{\rm A401}$: 0.010313, +/- 0.000627
$T_{\rm e, A401}$: 8.529603, +/- 0.258357
$A_{\rm D, A401}$: 331575.663144, +/- 271720.123229
$RA_{\rm A399}$: 44.463095, +/- 0.003259
$DEC_{\rm A399}$: 13.039885, +/- 0.003664
$\beta_{\rm A399}$: 0.901052, +/- 0.041325
$r_{\rm c, A399}$ [$^{\prime}$]: 3.371025, +/- 0.438960
$e_{\rm A399}$: 1.208112, +/- 0.115903
$\theta_{\rm A399}$: 117.734794, +/- 11.240510
$\tau_{\rm A399}$: 0.008260, +/- 0.000609
$T_{\rm e, A399}$: 7.171819, +/- 0.195523
$A_{\rm D, A399}$: 48473.830947, +/- 190209.667227
$RA_{\rm fil}: 44.680945, +/- 0.021254
$DEC_{\rm fil}: 13.362733, +/- 0.031378
$L_{\rm fil}$: 14.346114, +/- 2.274580
$W_{\rm fil}$: 12.977831, +/- 1.266832
$\tau_{\r

RuntimeError: latex was not able to process the following string:
b'$RA_{\\\\rm fil}'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpbiq3h776 7d7c1dc395e18181fbece161ce3831d8.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./7d7c1dc395e18181fbece161ce3831d8.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/u

Error in callback <function _draw_all_if_interactive at 0x7ea270545300> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: latex was not able to process the following string:
b'$RA_{\\\\rm fil}'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpx09s6tsr 7d7c1dc395e18181fbece161ce3831d8.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./7d7c1dc395e18181fbece161ce3831d8.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/u

RuntimeError: latex was not able to process the following string:
b'$RA_{\\\\rm fil}'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpum_azkgl 7d7c1dc395e18181fbece161ce3831d8.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./7d7c1dc395e18181fbece161ce3831d8.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/share/texmf/tex/latex/cm-super/type1ec.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/u

<Figure size 5590x5590 with 676 Axes>

In [ ]:
Number of iterations: 13656
$RA_{\rm A401}$: 44.741279, +/- 0.001608
$DEC_{\rm A401}$: 13.577833, +/- 0.002167
$\beta_{\rm A401}$: 1.081222, +/- 0.075397
$r_{\rm c, A401}$ [$^{\prime}$]: 3.152839, +/- 0.395240
$e_{\rm A401}$: 1.324916, +/- 0.117789
$\theta_{\rm A401}$: 109.795009, +/- 6.667116
$\tau_{\rm A401}$: 0.010361, +/- 0.000716
$T_{\rm e, A401}$: 8.495815, +/- 0.245556
$A_{\rm D, A401}$: 97293.346391, +/- 200132.274262
$RA_{\rm A399}$: 44.463061, +/- 0.003117
$DEC_{\rm A399}$: 13.039613, +/- 0.003519
$\beta_{\rm A399}$: 0.936454, +/- 0.065619
$r_{\rm c, A399}$ [$^{\prime}$]: 3.098175, +/- 0.462653
$e_{\rm A399}$: 1.157243, +/- 0.125563
$\theta_{\rm A399}$: 116.608116, +/- 14.004839
$\tau_{\rm A399}$: 0.008372, +/- 0.000674
$T_{\rm e, A399}$: 7.182108, +/- 0.190611
$A_{\rm D, A399}$: 5184.614229, +/- 140825.740495
$RA_{\rm fil}: 44.672408, +/- 0.019142
$DEC_{\rm fil}: 13.341570, +/- 0.027347
$L_{\rm fil}$: 14.581966, +/- 1.939244
$W_{\rm fil}$: 12.024050, +/- 1.127387
$\tau_{\rm fil}$: 0.001331, +/- 0.000172
$T_{\rm e, fil}$: 6.537729, +/- 0.350609
$A_{\rm D, fil}$: 778.267073, +/- 63033.310746
$v_{r, avg}$: 2497.934170, +/- 312.632446

In [ ]:
Number of iterations: 16662
$RA_{\rm A401}$: 44.741065, +/- 0.001514
$DEC_{\rm A401}$: 13.578850, +/- 0.002066
$\beta_{\rm A401}$: 1.094965, +/- 0.074308
$r_{\rm c, A401}$ [$^{\prime}$]: 3.136663, +/- 0.375450
$e_{\rm A401}$: 1.337288, +/- 0.106835
$\theta_{\rm A401}$: 109.929819, +/- 6.220435
$\tau_{\rm A401}$: 0.010449, +/- 0.000679
$T_{\rm e, A401}$: 8.525355, +/- 0.248107
$A_{\rm D, A401}$: 86092.623599, +/- 188450.700481
$RA_{\rm A399}$: 44.463598, +/- 0.002966
$DEC_{\rm A399}$: 13.038696, +/- 0.003240
$\beta_{\rm A399}$: 1.053455, +/- 0.239253
$r_{\rm c, A399}$ [$^{\prime}$]: 3.469339, +/- 0.587106
$e_{\rm A399}$: 1.162149, +/- 0.123363
$\theta_{\rm A399}$: 111.713122, +/- 14.681799
$\tau_{\rm A399}$: 0.008354, +/- 0.000845
$T_{\rm e, A399}$: 7.224879, +/- 0.182117
$A_{\rm D, A399}$: 8855.587065, +/- 152083.536100
$RA_{\rm fil}: 44.666543, +/- 0.037024
$DEC_{\rm fil}: 13.341406, +/- 0.056074
$L_{\rm fil}$: 15.514648, +/- 4.528076
$W_{\rm fil}$: 12.760684, +/- 1.144964
$\tau_{\rm fil}$: 0.001368, +/- 0.000163
$T_{\rm e, fil}$: 6.343114, +/- 0.344954
$A_{\rm D, fil}$: 520.022570, +/- 45837.267548
$v_{r, avg}$: 2552.610171, +/- 313.807366

data_orig_98 = enmap.read_map(")

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable


from astropy.visualization import (ZScaleInterval,
                                    PercentileInterval,
                                    MinMaxInterval,
                                    ImageNormalize,
                                    simple_norm)
from matplotlib.ticker import MaxNLocator

region_center_ra = 44.5916744
region_center_dec = 13.2999979

box_region = ut.get_region(region_center_ra, region_center_dec, 3.1 * 1.9)

data_map_pl = ut.imap_dim_check(enmap.read_map("/home/gill/research/ACT/bridge/data_paper/data/data/act_no_reproj/act_cut_dr6v2_pa5_f098_4way_coadd_map_srcfree.fits",
                                               box=box_region))

flux_factor = ut.flux_factor("pa5", 98) # (Jy/sr/uK)

data_map_pl *= flux_factor / 1000

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection=data_map_pl.wcs)

norm = ImageNormalize(data_map_pl, interval=ZScaleInterval())

im = ax.imshow(data_map_pl, origin='lower', cmap='viridis', norm=norm, interpolation='none')
# Access coordinate axes
ra = ax.coords[0]
dec = ax.coords[1]
color = 'white'

x_center = region_center_ra
y_center = region_center_dec

# Set labels
ra.set_axislabel('Right ascension')
dec.set_axislabel('Declination')

# Ensure tick labels are in float degrees
ra.set_major_formatter('d')
dec.set_major_formatter('d')

ax.invert_xaxis()

divider = make_axes_locatable(ax)

cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.046, pad=0.04, label='kJy/sr')
# cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=10))

# make boxes of 1.9 deg x 1.9 deg around the central region and around it
import matplotlib.patches as patches

box_size = 1.9  # size of each box in degrees
# Create a 3x3 grid of boxes centered on (x_center, y_center) and label them
region_counter = 0

for dy_offset in [box_size, 0, -box_size]:
    for dx_offset in [box_size, 0, -box_size]:
        lower_left = (x_center + dx_offset - box_size/2, y_center + dy_offset - box_size/2)
        rect = patches.Rectangle(lower_left, box_size, box_size, 
                                 edgecolor=color, facecolor='none', lw=1,
                                 transform=ax.get_transform('world'))
        ax.add_patch(rect)
        
        label_x = lower_left[0] + box_size/2
        label_y = lower_left[1] + 0.8 * box_size
        
        if dx_offset == 0 and dy_offset == 0:
            label = "Cluster pair"
        else:
            label = f"Region {region_counter}"
            region_counter += 1
        
        ax.text(label_x, label_y, label, color=color, fontsize=14,
                ha='center', va='bottom', transform=ax.get_transform('world'))

plt.savefig("../plots/region_map.eps", bbox_inches='tight', dpi=300, format='eps')
plt.show()

In [ ]:
cax = divider.append_axes("bottom", size="5%", pad=0.2)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal', label='kJy/sr')
cbar.ax.xaxis.set_major_locator(MaxNLocator(nbins=10))

In [ ]:
data_orig_98 = ut.imap_dim_check(enmap.read_map("/home/gill/research/ACT/bridge/data_paper/data/data/20210728/dr6v2_pa5_f090_4way_set1_map_srcfree.fits"))
data_fit_98 = ut.imap_dim_check(enmap.read_map("/home/gill/research/ACT/bridge/data_paper/data/data/act_no_reproj/act_cut_dr6v2_pa5_f098_4way_set1_map_srcfree.fits"))

ut.plot_image(data_orig_98 - data_fit_98, interval_type="zscale")
ut.plot_image(data_fit_98)

ratio =  data_fit_98 / data_orig_98
print(np.nanmean(ratio), np.nanstd(ratio))

In [ ]:
data_orig_98.shape